In [8]:
import requests
from ipyleaflet import Map, ImageOverlay, projections, ImageService, basemaps,  WidgetControl
from ipywidgets import SelectionSlider, Layout, Label, VBox, Dropdown
from datetime import datetime, timezone
import time
import json
import urllib.parse
from ipywidgets import Output, HTML

# Working with Tropospheric Emissions: Monitoring of Pollution (TEMPO)'s Image Service in ipyleaflet

## DEPRECATED: Export Image with Custom Color Ramp and Image Overlay

In [ ]:

def convert_to_milliseconds(date_time_str):
    """Converts a date-time string in 'YYYY-MM-DD HH:MM:SS' format to milliseconds since epoch."""
    dt = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    milliseconds_since_epoch = int(dt.timestamp() * 1000)

    return milliseconds_since_epoch


# Function to get the JSON request of the image using the image export service
def response_export_image(image_service_url, bbox, date_time):
    
    """For more info on how to adjuste parameters, visit https://developers.arcgis.com/rest/services-reference/enterprise/export-image/"""
   
    # Provide a rendering rule for the color ramp, with a custom ColorRamp
    rendering_rule = {
        "rasterFunctionArguments": {
            "colorRamp": { #custom ColorRamp
                "type": "multipart",
                "colorRamps": [ 
                    {
                        "type": "algorithmic",
                        "fromColor": [0, 0, 255, 255],
                        "toColor": [0, 255, 255, 255],
                        "algorithm": "esriCIELabAlgorithm"
                    },
                    {
                        "type": "algorithmic",
                        "fromColor": [0, 255, 255, 255],
                        "toColor": [255, 255, 0, 255],
                        "algorithm": "esriCIELabAlgorithm"
                    },
                    {
                        "type": "algorithmic",
                        "fromColor": [255, 255, 0, 255],
                        "toColor": [255, 0, 0, 255],
                        "algorithm": "esriCIELabAlgorithm"
                    }
                ]
            },
            "Raster": {
                "rasterFunctionArguments": {
                    "StretchType": 5,
                    "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]], # min value is 0, max value is 3e+16
                    "DRA": False,
                    "UseGamma": False,
                    "Gamma": [1],
                    "ComputeGamma": True,
                    "Min": 0,
                    "Max": 255
                },
                "rasterFunction": "Stretch",
                "outputPixelType": "U64", # must coincide with parameter's pixel type
                "variableName": "Raster"
            }
        },
        "rasterFunction": "Colormap",
        "variableName": "Raster"
    }

    # Convert rendering rule to JSON string
    rendering_rule_json = json.dumps(rendering_rule)

    
    params = {
        'bbox': ','.join(map(str, bbox)),
        'bboxSR': '4326', #4326
        'imageSR': '4326',#4326
        'size': '1000,1000',
        'time': '',
        'format': 'jpgpng',
        'pixelType': 'U64', # must coincide with rendering rule's pixel type or leave blank
        'noData': '',
        'noDataInterpretation': 'esriNoDataMatchAny',
        'interpolation': 'RSP_BilinearInterpolation',
        'compression': '',
        'compressionQuality': '',
        'bandIds': '',
        'sliceId': '1000',
        'mosaicRule': '',
        'renderingRule': rendering_rule_json,
        'adjustAspectRatio': 'true',
        'validateExtent': 'false',
        'lercVersion': '',
        'compressionTolerance': '',
        'f': 'json',
        'variableName': 'NO2 Troposphere'
    }
    response = requests.get(image_service_url, params=params)
    response.raise_for_status()
    print(response.json())
    return response.json()


image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/exportImage"

# User inputs defaults
date_time_str = "2024-07-10 9:16:57" #EST time #TODO
bbox = [-91, 20, -67, 44]
print(bbox)
# -85.051129


# Convert time to milliseconds
date_time = convert_to_milliseconds(date_time_str) # equals 1720617417000 in ms since epoch

# retrieves images given user input
response = response_export_image(image_service_url, bbox, date_time)


xmin , ymin, xmax, ymax = response['extent']['xmin'] ,  response['extent']['ymin'] ,  response['extent']['xmax'] ,  response['extent']['ymax']

center_lat = (ymin + ymax) / 2
center_lon = (xmin + xmax) / 2



# Initialize the map
m = Map(center=(center_lat, center_lon), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

# Add new image layer

bounds = [[ymin, xmin], [ymax, xmax]]

print(bounds)

image_overlay = ImageOverlay(url=response['href'], bounds=bounds, opacity=0.5)

m.add(image_overlay)



m


## Image Service and TEMPO's colormap YELLOW

In [9]:


def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone.utc)
    date_time_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date_time_str




def on_click(**kwargs):
    if kwargs.get('type') == 'click':
        print(str(kwargs.get('coordinates')))
    
    if kwargs.get('type') == 'mousemove':
        latlng = kwargs.get('coordinates')
        lat, lng = latlng
        coordinates_label.value = f"Coordinates: ({lat:.5f}, {lng:.5f})"
    

# The actual start times of observations from the datafiles
time_values = [
    1715683263000,
    1715685668000,
    1715688073000,
    1715690478000,
    1715692883000,
    1715695288000,
    1715698888000,
    1715702488000,
    1715706088000,
    1715709688000,
    1715713288000,
    1715716888000,
    1715720488000,
    1715724088000,
    1715726493000,
    1715728898000,
    1715731303000,
    1715733708000,
]

image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_tile_cache_test/ImageServer"

# Initialize the map
m = Map(center=(47,-122), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

tempo_image_service = ImageService(url=image_service_url, 
                                   rendering_rule={"rasterFunction":"RGB_Colormap_Yellow"}, 
                                   format="jpgpng",
                                   opacity=0.5)


#creating a list with the UTC times with time_values for easy visualization of time
time_strings = [convert_from_milliseconds(t) for t in time_values]  

#creating a list of tuples to input in SelectionSlider's options for easy visualization of time
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

#creating the slider
slider = SelectionSlider(description='Time:', options=time_options, layout=Layout(width='700px', height='20px'))

#creating a Label for the VBox
time_label = Label(value='Time Slider')

#A handler that will update the map everytime the user moves the slider
def update_image(change):

    tempo_image_service.time = [change.new,1715733708000]
    
#Listens to the slider's user input and helps update the map
slider.observe(update_image, 'value')

#creates a VBox to contain the slider and be placed in the map
vbox = VBox([slider, time_label])

#slider placed in bottomleft of the map
control = WidgetControl(widget=vbox, position='bottomleft')

# Output widget to listen to the user's mouse hovering over the map
output = Output()
controloutput = WidgetControl(widget=output, position='topright')

# Label widget to display coordinates
coordinates_label = HTML(value="Coordinates: ")
coordinates_control = WidgetControl(widget=coordinates_label, position='bottomright')

#add all widgets to the map
m.add(tempo_image_service)
m.add(control)
m.add(controloutput)
m.add(coordinates_control)

# when user hovers over the map coordinates_label gets updated and prints the coordinates where clicked
m.on_interaction(on_click)
m

Map(center=[47, -122], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Image Service and TEMPO's colormap TORCH

In [10]:


def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone.utc)
    date_time_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date_time_str


def on_click(**kwargs):
    if kwargs.get('type') == 'click':
        print(str(kwargs.get('coordinates')))
    
    if kwargs.get('type') == 'mousemove':
        latlng = kwargs.get('coordinates')
        lat, lng = latlng
        coordinates_label.value = f"Coordinates: ({lat:.5f}, {lng:.5f})"
    

# The actual start times of observations from the datafiles
time_values = [
    1715683263000,
    1715685668000,
    1715688073000,
    1715690478000,
    1715692883000,
    1715695288000,
    1715698888000,
    1715702488000,
    1715706088000,
    1715709688000,
    1715713288000,
    1715716888000,
    1715720488000,
    1715724088000,
    1715726493000,
    1715728898000,
    1715731303000,
    1715733708000,
]

image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_tile_cache_test/ImageServer"

# Initialize the map
m = Map(center=(47,-122), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

tempo_image_service = ImageService(url=image_service_url, 
                                   rendering_rule={"rasterFunction":"RGB_Colormap_Torch"}, 
                                   format="jpgpng",
                                   opacity=0.5)


#creating a list with the UTC times with time_values for easy visualization of time
time_strings = [convert_from_milliseconds(t) for t in time_values]  

#creating a list of tuples to input in SelectionSlider's options for easy visualization of time
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

#creating the slider
slider = SelectionSlider(description='Time:', options=time_options, layout=Layout(width='700px', height='20px'))

#creating a Label for the VBox
time_label = Label(value='Time Slider')

#A handler that will update the map everytime the user moves the slider
def update_image(change):

    tempo_image_service.time = [change.new,1715733708000]
    
#Listens to the slider's user input and helps update the map
slider.observe(update_image, 'value')

#creates a VBox to contain the slider and be placed in the map
vbox = VBox([slider, time_label])

#slider placed in bottomleft of the map
control = WidgetControl(widget=vbox, position='bottomleft')

# Output widget to listen to the user's mouse hovering over the map
output = Output()
controloutput = WidgetControl(widget=output, position='topright')

# Label widget to display coordinates
coordinates_label = HTML(value="Coordinates: ")
coordinates_control = WidgetControl(widget=coordinates_label, position='bottomright')

#add all widgets to the map
m.add(tempo_image_service)
m.add(control)
m.add(controloutput)
m.add(coordinates_control)

# when user hovers over the map coordinates_label gets updated and prints the coordinates where clicked
m.on_interaction(on_click)
m

Map(center=[47, -122], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Image Service and Custom Color Ramp

In [4]:


def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone.utc)
    date_time_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date_time_str




def on_click(**kwargs):
    if kwargs.get('type') == 'click':
        print(str(kwargs.get('coordinates')))
    
    if kwargs.get('type') == 'mousemove':
        latlng = kwargs.get('coordinates')
        lat, lng = latlng
        coordinates_label.value = f"Coordinates: ({lat:.5f}, {lng:.5f})"
    

# The actual start times of observations from the datafiles
time_values = [
    1715683263000,
    1715685668000,
    1715688073000,
    1715690478000,
    1715692883000,
    1715695288000,
    1715698888000,
    1715702488000,
    1715706088000,
    1715709688000,
    1715713288000,
    1715716888000,
    1715720488000,
    1715724088000,
    1715726493000,
    1715728898000,
    1715731303000,
    1715733708000,
]


rendering_rule = {
    "rasterFunctionArguments": {
        "colorRamp": { #custom ColorRamp. WARNING: colormap rendering rule must be as small as possible due to URL lenght limitations
            "type": "multipart",
            "colorRamps": [ 
                {
                    "type": "algorithmic",
                    "fromColor": [0, 0, 255, 255],
                    "toColor": [0, 255, 255, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [0, 255, 255, 255],
                    "toColor": [255, 255, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                },
                {
                    "type": "algorithmic",
                    "fromColor": [255, 255, 0, 255],
                    "toColor": [255, 0, 0, 255],
                    "algorithm": "esriCIELabAlgorithm"
                }
            ]
        },
        "Raster": {
            "rasterFunctionArguments": {
                "StretchType": 5,
                "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]], # min value is 0, max value is 3e+16
                "DRA": False,
                "UseGamma": False,
                "Gamma": [1],
                "ComputeGamma": True,
                "Min": 0,
                "Max": 255
            },
            "rasterFunction": "Stretch",
            "outputPixelType": "U64", # must coincide with parameter's pixel type
            "variableName": "Raster"
        }
    },
    "rasterFunction": "Colormap",
    "variableName": "Raster"
}

image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/"

# Initialize the map
m = Map(center=(47,-122), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

tempo_image_service = ImageService(url=image_service_url, 
                                   rendering_rule=rendering_rule, 
                                   format="jpgpng",
                                   opacity=0.5)


#creating a list with the UTC times with time_values for easy visualization of time
time_strings = [convert_from_milliseconds(t) for t in time_values]  

#creating a list of tuples to input in SelectionSlider's options for easy visualization of time
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

#creating the slider
slider = SelectionSlider(description='Time:', options=time_options, layout=Layout(width='700px', height='20px'))

#creating a Label for the VBox
time_label = Label(value='Time Slider')

#A handler that will update the map everytime the user moves the slider
def update_image(change):

    tempo_image_service.time = [change.new,1715733708000]
    
#Listens to the slider's user input and helps update the map
slider.observe(update_image, 'value')

#creates a VBox to contain the slider and be placed in the map
vbox = VBox([slider, time_label])

#slider placed in bottomleft of the map
control = WidgetControl(widget=vbox, position='bottomleft')

# Output widget to listen to the user's mouse hovering over the map
output = Output()
controloutput = WidgetControl(widget=output, position='topright')

# Label widget to display coordinates
coordinates_label = HTML(value="Coordinates: ")
coordinates_control = WidgetControl(widget=coordinates_label, position='bottomright')

#add all widgets to the map
m.add(tempo_image_service)
m.add(control)
m.add(controloutput)
m.add(coordinates_control)

# when user hovers over the map coordinates_label gets updated and prints the coordinates where clicked
m.on_interaction(on_click)
m

Map(center=[47, -122], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Image Service and ESRI's colormap template

In [5]:


def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone.utc)
    date_time_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date_time_str


def on_click(**kwargs):
    if kwargs.get('type') == 'click':
        print(str(kwargs.get('coordinates')))
    
    if kwargs.get('type') == 'mousemove':
        latlng = kwargs.get('coordinates')
        lat, lng = latlng
        coordinates_label.value = f"Coordinates: ({lat:.5f}, {lng:.5f})"
    

# The actual start times of observations from the datafiles
time_values = [
    1715683263000,
    1715685668000,
    1715688073000,
    1715690478000,
    1715692883000,
    1715695288000,
    1715698888000,
    1715702488000,
    1715706088000,
    1715709688000,
    1715713288000,
    1715716888000,
    1715720488000,
    1715724088000,
    1715726493000,
    1715728898000,
    1715731303000,
    1715733708000,
]


rendering_rule = {
        "rasterFunctionArguments": {
            "ColorrampName": "Temperature",  #preset ColorRamp from ArcGIS
            "Raster": {
                "rasterFunctionArguments": {
                    "StretchType": 5,
                    "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]], # min value is 0, max value is 3e+16
                    "DRA": False,
                    "UseGamma": False,
                    "Gamma": [1],
                    "ComputeGamma": True,
                    "Min": 0,
                    "Max": 255
                },
                "rasterFunction": "Stretch",
                "outputPixelType": "U64", # must coincide with parameter's pixel type
                "variableName": "Raster"
            }
        },
        "rasterFunction": "Colormap",
        "variableName": "Raster"
    }

image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/"

# Initialize the map
m = Map(center=(47,-122), zoom=3, basemap=basemaps.Esri.WorldTopoMap)

tempo_image_service = ImageService(url=image_service_url, 
                                   rendering_rule=rendering_rule, 
                                   format="jpgpng",
                                   opacity=0.5)


#creating a list with the UTC times with time_values for easy visualization of time
time_strings = [convert_from_milliseconds(t) for t in time_values]  

#creating a list of tuples to input in SelectionSlider's options for easy visualization of time
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

#creating the slider
slider = SelectionSlider(description='Time:', options=time_options, layout=Layout(width='700px', height='20px'))

#creating a Label for the VBox
time_label = Label(value='Time Slider')

#A handler that will update the map everytime the user moves the slider
def update_image(change):

    tempo_image_service.time = [change.new,1715733708000]
    
#Listens to the slider's user input and helps update the map
slider.observe(update_image, 'value')

#creates a VBox to contain the slider and be placed in the map
vbox = VBox([slider, time_label])

#slider placed in bottomleft of the map
control = WidgetControl(widget=vbox, position='bottomleft')

# Output widget to listen to the user's mouse hovering over the map
output = Output()
controloutput = WidgetControl(widget=output, position='topright')

# Label widget to display coordinates
coordinates_label = HTML(value="Coordinates: ")
coordinates_control = WidgetControl(widget=coordinates_label, position='bottomright')

#add all widgets to the map
m.add(tempo_image_service)
m.add(control)
m.add(controloutput)
m.add(coordinates_control)

# when user hovers over the map coordinates_label gets updated and prints the coordinates where clicked
m.on_interaction(on_click)
m

Map(center=[47, -122], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…